In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.6 MB/s eta 0:00:00


#### 1. Generación del Dataset de Rendiciones de Cuenta

In [2]:
import pandas as pd
from faker import Faker
import numpy as np
import random
from datetime import datetime, timedelta

# 1. Generación del Dataset de Rendiciones de Cuenta
seed = 42
random.seed(seed)
np.random.seed(seed)
num_rendiciones = 30
fake = Faker('es_AR')
data_rendiciones = {
    'ID_Rendicion': range(1, num_rendiciones + 1),
    'Fecha_Presentacion': pd.to_datetime([fake.date_between(start_date='-365d', end_date='today') for _ in range(num_rendiciones)]),
    'Empleado': [fake.name() for _ in range(num_rendiciones)],
    'Departamento': [random.choice(['Ventas', 'Marketing', 'Compras', 'RRHH', 'Finanzas']) for _ in range(num_rendiciones)],
    'Tipo_Gasto': [random.choice(['Viáticos', 'Material de Oficina', 'Almuerzo', 'Transporte', 'Otros']) for _ in range(num_rendiciones)],
    'Descripcion': [fake.random_element(elements=('gastos_generales', 'viaticos', 'gastos_comida', 'otros')) for _ in range(num_rendiciones)], #Correción aquí
    'Monto': np.random.uniform(1000, 50000, num_rendiciones).round(2),
    'Justificante_Adjunto': random.choices(['Sí', 'No'], weights=[0.8, 0.2], k=num_rendiciones),
    'Estado_Aprobacion': [random.choice(['Aprobado', 'Pendiente', 'Rechazado']) for _ in range(num_rendiciones)],
    'Es_Sospechoso': np.zeros(num_rendiciones, dtype=int)
}

df_rendiciones = pd.DataFrame(data_rendiciones)

# Función para marcar como sospechoso si el estado de aprobación es "Rechazado"
def marcar_sospechoso(row):
    if row['Estado_Aprobacion'] == 'Rechazado':
        return 1
    else:
        return 0

# Aplica la función a cada fila del DataFrame para crear la columna 'Es_Sospechoso'
df_rendiciones['Es_Sospechoso'] = df_rendiciones.apply(marcar_sospechoso, axis=1)

nombre_archivo_csv = 'df_rendiciones2.csv'
df_rendiciones.to_csv(nombre_archivo_csv, index=False)

print(f"El DataFrame se ha guardado exitosamente en el archivo '{nombre_archivo_csv}'")
df_rendiciones

El DataFrame se ha guardado exitosamente en el archivo 'df_rendiciones2.csv'


,ID_Rendicion,Fecha_Presentacion,Empleado,Departamento,Tipo_Gasto,Descripcion,Monto,Justificante_Adjunto,Estado_Aprobacion,Es_Sospechoso
0,1,2025-02-08,Sr(a). Juliana Diaz,Ventas,Almuerzo,gastos_generales,19352.47,No,Pendiente,0
1,2,2025-11-07,Francisco Santiago Moyano Gutierrez,Ventas,Almuerzo,gastos_comida,47585.00,No,Pendiente,0
2,3,2025-05-01,Santino Peralta Juarez,Compras,Material de Oficina,viaticos,36867.70,Sí,Rechazado,1
3,4,2025-03-15,Dr(a). Martín Olivera,Marketing,Material de Oficina,otros,30334.27,Sí,Pendiente,0
4,5,2025-04-13,Amparo Josefina Rios,Marketing,Almuerzo,gastos_comida,8644.91,No,Aprobado,0
5,6,2025-12-17,Francesca Duarte,Marketing,Viáticos,gastos_generales,8643.73,Sí,Pendiente,0
6,7,2025-08-29,Sr(a). Lucio Torres,Ventas,Viáticos,viaticos,3846.10,Sí,Aprobado,0
7,8,2025-08-11,Ignacio Ponce,Finanzas,Transporte,otros,43442.63,Sí,Aprobado,0
8,9,2025-11-10,Juana Lorenzo Perez,Ventas,Viáticos,otros,30454.64,Sí,Rechazado,1
9,10,2025-12-02,Emiliano Miranda Garcia,Finanzas,Almuerzo,gastos_comida,35695.56,Sí,Rechazado,1


### Algoritmo rendicion de cuentas2

In [3]:
import pandas as pd
from faker import Faker
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


data_rendiciones = pd.read_csv('df_rendiciones2.csv')
df_rendiciones = pd.DataFrame(data_rendiciones)

# 1. Simulación de un Sistema Existente (Ejemplo: Sistema de Gestión de Gastos)
class SistemaGestionGastos:
    def __init__(self):
        self.rendiciones = pd.DataFrame()
        self.empleados = {} # Simulación de información de empleados

    def agregar_empleado(self, id_empleado, nombre):
        self.empleados[id_empleado] = {'nombre': nombre}

    def registrar_rendicion(self, id_rendicion, id_empleado, fecha, tipo_gasto, monto, descripcion, justificante='Sí', estado='Pendiente'):  # Añadir 'estado'
        nueva_rendicion = pd.DataFrame([{
            'ID_Rendicion': id_rendicion,
            'ID_Empleado': id_empleado,
            'Fecha': fecha,
            'Tipo_Gasto': tipo_gasto,
            'Monto': monto,
            'Descripcion': descripcion,
            'Justificante_Adjunto': justificante,
            'Estado': estado,  # Usar el estado por defecto o el proporcionado
            'Es_Sospechosa': 0  # Inicialmente no sospechosa
        }])
        self.rendiciones = pd.concat([self.rendiciones, nueva_rendicion], ignore_index=True)
        print(f"Rendición {id_rendicion} registrada para empleado {id_empleado}.")

    def obtener_rendiciones(self):
        return self.rendiciones

    def actualizar_estado_rendicion(self, id_rendicion, nuevo_estado):
        if not self.rendiciones.empty and id_rendicion in self.rendiciones['ID_Rendicion'].values:
            self.rendiciones.loc[self.rendiciones['ID_Rendicion'] == id_rendicion, 'Estado'] = nuevo_estado
            print(f"Estado de rendición {id_rendicion} actualizado a '{nuevo_estado}'.")
        else:
            print(f"Rendición con ID {id_rendicion} no encontrada.")

    def mostrar_rendiciones_sospechosas(self):
        sospechosas = self.rendiciones[self.rendiciones['Es_Sospechosa'] == 1]
        if not sospechosas.empty:
            print("\nRendiciones de Cuenta Marcadas como Sospechosas:")
            print(sospechosas[['ID_Rendicion', 'ID_Empleado', 'Fecha', 'Tipo_Gasto', 'Monto', 'Descripcion', 'Justificante_Adjunto', 'Estado', 'Es_Sospechosa']])  # Incluir Es_Sospechosa
        else:
            print("\nNo se encontraron rendiciones sospechosas.")

# 2. Simulación de un Algoritmo de Detección de Fraude (Ejemplo: Reglas Simples)
def detectar_fraude_simple(rendiciones_df):
    sospechosas_indices = []
    for index, row in rendiciones_df.iterrows():
        if row['Monto'] > 1000 and row['Tipo_Gasto'] == 'Otros':
            sospechosas_indices.append(index)
        elif row['Monto'] > 500 and row['Justificante_Adjunto'] == 'No':
            sospechosas_indices.append(index)
        elif 'viaje' in row['Descripcion'].lower() and row['Monto'] > 300:
            sospechosas_indices.append(index)
    rendiciones_df.loc[rendiciones_df.index.isin(sospechosas_indices), 'Es_Sospechosa'] = 1  # Corregido para usar isin
    return rendiciones_df

# 3. Simulación de un Modelo de Machine Learning (Entrenamiento Básico)
def entrenar_modelo_deteccion_fraude(df):
    #Prepares Dataframe from CSV
    df = df.fillna('Desconocido')
    df['Justificante_Adjunto'] = df['Justificante_Adjunto'].astype(str)
    df['Monto'] = pd.to_numeric(df['Monto'], errors='coerce')
    df = df.dropna(subset=['Monto'])
    df['Es_Sospechoso'] = df['Es_Sospechoso'].astype(int)
    df = df.drop(columns=['ID_Rendicion', 'Empleado', 'Fecha_Presentacion', 'Descripcion', 'Estado_Aprobacion'])

    X_train = df[['Monto', 'Tipo_Gasto', 'Justificante_Adjunto']]
    y_train = df['Es_Sospechoso']

    # Codificación de variables categóricas
    le_tipo = LabelEncoder()
    X_train['Tipo_Gasto'] = le_tipo.fit_transform(X_train['Tipo_Gasto'])
    le_justificante = LabelEncoder()
    X_train['Justificante_Adjunto'] = le_justificante.fit_transform(X_train['Justificante_Adjunto'])

    modelo = RandomForestClassifier(random_state=42)
    modelo.fit(X_train, y_train)
    return modelo, le_tipo, le_justificante

def aplicar_modelo_ml(rendiciones_df, modelo, le_tipo, le_justificante):
    if rendiciones_df.empty:
        return rendiciones_df

    if not all(col in rendiciones_df.columns for col in ['Monto', 'Tipo_Gasto', 'Justificante_Adjunto']):
        print("Advertencia: Las columnas necesarias para la predicción no están presentes en el DataFrame.")
        return rendiciones_df

    X_pred = rendiciones_df[['Monto', 'Tipo_Gasto', 'Justificante_Adjunto']].copy()

    # Manejo de valores faltantes (si los hubiera)
    for col in X_pred.columns:
        if X_pred[col].isnull().any():
            print(f"Advertencia: Valores nulos encontrados en la columna '{col}'. Imputando con 'Desconocido' (para categóricas) o 0 (para numéricas).")
            if X_pred[col].dtype == 'object':
                X_pred[col].fillna('Desconocido', inplace=True)
            else:
                X_pred[col].fillna(0, inplace=True)

    try:
        X_pred['Tipo_Gasto_Cod'] = le_tipo.transform(X_pred['Tipo_Gasto'])
    except ValueError as e:
        print(f"Error al codificar 'Tipo_Gasto': {e}")
        print("Categorías conocidas:", le_tipo.classes_)
        print("Categorías problemáticas:", set(X_pred['Tipo_Gasto'].unique()) - set(le_tipo.classes_))
        return rendiciones_df
    try:
        X_pred['Justificante_Adjunto_Cod'] = le_justificante.transform(X_pred['Justificante_Adjunto'])
    except ValueError as e:
        print(f"Error al codificar 'Justificante_Adjunto': {e}")
        print("Categorías conocidas:", le_justificante.classes_)
        print("Categorías problemáticas:", set(X_pred['Justificante_Adjunto'].unique()) - set(le_justificante.classes_))
        return rendiciones_df

    X_pred_encoded = X_pred[['Monto', 'Tipo_Gasto_Cod', 'Justificante_Adjunto_Cod']]

    try:
        predicciones = modelo.predict(X_pred_encoded)
        rendiciones_df['Es_Sospechosa_ML'] = predicciones
        rendiciones_df['Es_Sospechosa'] = rendiciones_df['Es_Sospechosa'].astype(int) | rendiciones_df['Es_Sospechosa_ML'].astype(int)
    except ValueError as e:
        print(f"Error al predecir con el modelo: {e}")
        print("Verifica que las columnas y tipos de datos sean correctos.")
    return rendiciones_df

# 4. Integración del Algoritmo en el Sistema Existente
print("\n9.2 Integración de Algoritmos en Sistemas Existentes")
print("\nSimulación de un Sistema de Gestión de Gastos y la Integración de Algoritmos para Detección de Fraude.")

# Inicializar el sistema de gestión de gastos
sistema_gastos = SistemaGestionGastos()
sistema_gastos.agregar_empleado(1, "Juan Pérez")
sistema_gastos.agregar_empleado(2, "María Gómez")

# Registrar algunas rendiciones de cuenta
sistema_gastos.registrar_rendicion(101, 1, datetime(2024, 10, 20), 'Viáticos', 750.00, "Viaje a Buenos Aires")
sistema_gastos.registrar_rendicion(102, 2, datetime(2024, 10, 21), 'Material de Oficina', 50.00, "Compra de bolígrafos")
sistema_gastos.registrar_rendicion(103, 1, datetime(2024, 10, 22), 'Otros', 1200.00, "Servicios externos", justificante='Sí')
sistema_gastos.registrar_rendicion(104, 2, datetime(2024, 10, 23), 'Almuerzo', 35.00, "Reunión con cliente")
sistema_gastos.registrar_rendicion(105, 1, datetime(2024, 10, 24), 'Otros', 650.00, "Gastos varios", justificante='No')
sistema_gastos.registrar_rendicion(106, 2, datetime(2024, 10, 25), 'Viáticos', 900.00, "Viaje a Córdoba")
sistema_gastos.registrar_rendicion(107, 1, datetime(2024, 10, 26), 'Material de Oficina', 15.00, "Compra de papel")
sistema_gastos.registrar_rendicion(108, 2, datetime(2024, 10, 27), 'Otros', 400.00, "Suministros")
sistema_gastos.registrar_rendicion(109, 1, datetime(2024, 10, 28), 'Almuerzo', 180.00, "Equipo de trabajo")
sistema_gastos.registrar_rendicion(110, 2, datetime(2024, 10, 29), 'Viáticos', 250.00, "Visita")
sistema_gastos.registrar_rendicion(111, 1, datetime(2024, 10, 30), 'Otros', 1500.00, "Consultoría")
sistema_gastos.registrar_rendicion(112, 2, datetime(2024, 10, 31), 'Almuerzo', 70.00, "Reunión")
sistema_gastos.registrar_rendicion(113, 1, datetime(2024, 11, 1), 'Viáticos', 1000.00, "Viaje de capacitación")
sistema_gastos.registrar_rendicion(114, 2, datetime(2024, 11, 2), 'Material de Oficina', 25.00, "Compra")
sistema_gastos.registrar_rendicion(115, 1, datetime(2024, 11, 3), 'Otros', 300.00, "Gastos menores")
sistema_gastos.registrar_rendicion(116, 2, datetime(2024, 11, 4), 'Almuerzo', 45.00, "Almuerzo")
sistema_gastos.registrar_rendicion(117, 1, datetime(2024, 11, 5), 'Viáticos', 1100.00, "Viaje")
sistema_gastos.registrar_rendicion(118, 2, datetime(2024, 11, 6), 'Material de Oficina', 100.00, "Compra")
sistema_gastos.registrar_rendicion(119, 1, datetime(2024, 11, 7), 'Otros', 700.00, "Servicios")
sistema_gastos.registrar_rendicion(120, 2, datetime(2024, 11, 8), 'Almuerzo', 200.00, "Reunión")
sistema_gastos.registrar_rendicion(121, 1, datetime(2024, 11, 9), 'Viáticos', 500.00, "Viaje")
sistema_gastos.registrar_rendicion(122, 2, datetime(2024, 11, 10), 'Otros', 1800.00, "Consultoría")  # Posiblemente sospechoso

# Obtener las rendiciones registradas
rendiciones_df = sistema_gastos.obtener_rendiciones()
if not rendiciones_df.empty:
    # Asegurarse de que 'Justificante_Adjunto' tenga valores consistentes
    if 'Justificante_Adjunto' not in rendiciones_df.columns:
        rendiciones_df['Justificante_Adjunto'] = ['Sí'] * len(rendiciones_df)  # Default si no está

    # 5. Aplicación del Algoritmo de Detección de Fraude (Reglas Simples)
    print("\nAplicando Algoritmo de Detección de Fraude (Reglas Simples)...")
    rendiciones_con_deteccion = detectar_fraude_simple(rendiciones_df.copy())
    sistema_gastos.rendiciones = rendiciones_con_deteccion
    sistema_gastos.mostrar_rendiciones_sospechosas()

    # 6. Integración de un Modelo de Machine Learning
    print("\nIntegrando Modelo de Machine Learning para Detección de Fraude...")
    modelo_ml, le_tipo_gast, le_justif = entrenar_modelo_deteccion_fraude(data_rendiciones.copy())
    rendiciones_con_ml = aplicar_modelo_ml(rendiciones_df.copy(), modelo_ml, le_tipo_gast, le_justif)
    if 'Es_Sospechosa_ML' in rendiciones_con_ml.columns:
        sistema_gastos.rendiciones = rendiciones_con_ml
        print("\nRendiciones después de aplicar el Modelo de Machine Learning:")
        sistema_gastos.mostrar_rendiciones_sospechosas()
    else:
        print("\nNo se pudieron aplicar las predicciones del modelo de Machine Learning.")

else:
    print("\nNo se registraron rendiciones en el sistema.")


9.2 Integración de Algoritmos en Sistemas Existentes

Simulación de un Sistema de Gestión de Gastos y la Integración de Algoritmos para Detección de Fraude.
Rendición 101 registrada para empleado 1.
Rendición 102 registrada para empleado 2.
Rendición 103 registrada para empleado 1.
Rendición 104 registrada para empleado 2.
Rendición 105 registrada para empleado 1.
Rendición 106 registrada para empleado 2.
Rendición 107 registrada para empleado 1.
Rendición 108 registrada para empleado 2.
Rendición 109 registrada para empleado 1.
Rendición 110 registrada para empleado 2.
Rendición 111 registrada para empleado 1.
Rendición 112 registrada para empleado 2.
Rendición 113 registrada para empleado 1.
Rendición 114 registrada para empleado 2.
Rendición 115 registrada para empleado 1.
Rendición 116 registrada para empleado 2.
Rendición 117 registrada para empleado 1.
Rendición 118 registrada para empleado 2.
Rendición 119 registrada para empleado 1.
Rendición 120 registrada para empleado 2.
Re